# Create Chunks for the Description
# Multiple Chunks for the description
[
    {
        "content": "Passive marine flooding...",
        "layer_id": "passive_marine_flooding"
    },
    {
        "content": "Limitations include not...",
        "layer_id": "passive_marine_flooding"
    },
    {
        "content": "Uses 2-meter resolution DEMs...",
        "layer_id": "passive_marine_flooding"
    }
]

In [2]:
import os
import json
import re
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
openai_key = os.getenv("OPENAI_API_KEY")

In [4]:
documentation_path = "../../data/documentation.json"

In [5]:
documentation = json.load(open(documentation_path))

In [6]:
def create_chunks(layer_id, description, max_sentences=3):
    sentences = re.split(r'(?<=[.!?])\s+', description)
    chunks = []

    for i in range(0, len(sentences), 3):
        chunk = sentences[i:i+2]
        chunks.append({
            "content": " ".join(chunk),
            "layer_id": layer_id
        })

    return chunks

In [7]:
documentation

{'passive_marine_flooding': {'title': 'Passive Marine Flooding',
  'description': "Passive marine flooding identifies areas hydrologically connected to the ocean that would be inundated by sea level rise scenarios. Using a modified bathtub approach with DEMs and MHHW tidal datum, the model identifies coastal areas below specified sea level heights that have direct surface connections to marine waters. Water levels are shown as they would appear during Mean Higher High Water (MHHW), representing the average higher high water height of each tidal day. These areas experience direct marine inundation as sea levels rise, with floodwater arriving via surface flow paths from the ocean. The modeling uses 2-meter resolution DEMs derived from LiDAR data, with horizontal and vertical accuracies conforming to FEMA flood mapping standards. Limitations include not accounting for wave action, coastal erosion, or dynamic coastal processes that are important along Hawaii's active coastlines.",
  'base_

In [8]:
all_chunks = []
for key, value in documentation.items():
    layer_id = value["base_layer_name"]
    description = value["description"]
    chunks = create_chunks(layer_id, description, max_sentences=3)
    all_chunks.extend(chunks)

In [9]:
all_chunks

[{'content': 'Passive marine flooding identifies areas hydrologically connected to the ocean that would be inundated by sea level rise scenarios. Using a modified bathtub approach with DEMs and MHHW tidal datum, the model identifies coastal areas below specified sea level heights that have direct surface connections to marine waters.',
  'layer_id': 'CRC:HI_State_80prob_{scenario}ft_SCI'},
 {'content': 'These areas experience direct marine inundation as sea levels rise, with floodwater arriving via surface flow paths from the ocean. The modeling uses 2-meter resolution DEMs derived from LiDAR data, with horizontal and vertical accuracies conforming to FEMA flood mapping standards.',
  'layer_id': 'CRC:HI_State_80prob_{scenario}ft_SCI'},
 {'content': 'Low-lying area flooding identifies areas that are topographically below sea level rise scenarios but lack direct hydrological connections to the ocean. These areas may become flooded through indirect pathways such as subsurface connections

In [10]:
from openai import OpenAI

client = OpenAI()

# Function to create embeddings for the chunks
def embed_chunks(chunks):
    response = client.embeddings.create(
        input=chunks,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [11]:
for chunk in all_chunks:
    chunk['embedding'] = embed_chunks(chunk['content'])
all_chunks


[{'content': 'Passive marine flooding identifies areas hydrologically connected to the ocean that would be inundated by sea level rise scenarios. Using a modified bathtub approach with DEMs and MHHW tidal datum, the model identifies coastal areas below specified sea level heights that have direct surface connections to marine waters.',
  'layer_id': 'CRC:HI_State_80prob_{scenario}ft_SCI',
  'embedding': [0.006575725972652435,
   0.05096031725406647,
   0.06726562231779099,
   0.03470487520098686,
   -0.0186114851385355,
   -0.009530127979815006,
   0.008850740268826485,
   -0.03343335911631584,
   0.01576927676796913,
   0.05604637786746025,
   -0.011886170133948326,
   -0.0036711872089654207,
   -0.018225044012069702,
   0.0038799899630248547,
   0.011668018065392971,
   0.035627346485853195,
   -0.002371624344959855,
   -0.0014927833108231425,
   -0.007410936523228884,
   0.023809736594557762,
   0.0352533683180809,
   0.009523894637823105,
   -0.008701150305569172,
   0.015881469473

In [11]:
# Save chunks to a file
with open('doc_chunks.json', 'w') as f:
    json.dump(all_chunks, f)

## Insert chunks into vector database (pgvector)

In [13]:
# Read chunks from file

with open('outputs/doc_chunks.json', 'r') as f:
    all_chunks = json.load(f)


In [14]:
import psycopg2
import numpy as np

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="climate_viewer_dev",
    user="dev_user",
    password="dev_password"
)
cur = conn.cursor()

# Create a table for the chunks
cur.execute("""
    CREATE TABLE IF NOT EXISTS public.chunks (
        id SERIAL PRIMARY KEY,
        layer_id VARCHAR(255) NOT NULL,
        content TEXT NOT NULL,
        embedding VECTOR(1536) NOT NULL
    )
""")

# List all tables in the database
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
tables = cur.fetchall()
print(tables)

# List all columns in the chunks table
cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name='chunks'")
columns = cur.fetchall()
print(columns)

# List all rows in the chunks table
cur.execute(query="SELECT * FROM public.chunks")

[('chunks',), ('rag_chunks',)]
[('id',), ('embedding',), ('created_at',), ('layer_id',), ('content',)]


In [15]:
# Insert chunks into the table
for chunk in all_chunks:
    cur.execute("""
        INSERT INTO chunks (layer_id, content, embedding)
        VALUES (%s, %s, %s)
    """, (chunk['layer_id'], chunk['content'], chunk['embedding']))


In [16]:
cur.execute(query="SELECT * FROM public.chunks")
print(cur.fetchall())

[(481, 'CRC:HI_State_80prob_{scenario}ft_SCI', 'Passive marine flooding identifies areas hydrologically connected to the ocean that would be inundated by sea level rise scenarios. Using a modified bathtub approach with DEMs and MHHW tidal datum, the model identifies coastal areas below specified sea level heights that have direct surface connections to marine waters.', '[0.006575726,0.050960317,0.06726562,0.034704875,-0.018611485,-0.009530128,0.00885074,-0.03343336,0.015769277,0.056046378,-0.01188617,-0.0036711872,-0.018225044,0.00387999,0.011668018,0.035627346,-0.0023716243,-0.0014927833,-0.0074109365,0.023809737,0.03525337,0.009523895,-0.00870115,0.01588147,-0.018661348,-0.023398364,0.0045967754,0.034555282,0.022002192,0.045475353,0.044153973,-0.025380433,-0.044951785,0.023797272,0.06751494,0.053303894,-0.04218437,0.04846715,-0.041336697,0.05068607,-0.020169714,-0.006195518,-0.026352767,0.059885852,0.012434666,-0.007853474,0.04557508,0.005257464,0.0064386018,0.031588417,0.029818268,-

## QA Testing

In [17]:
# Test Vector Search
user_query = "What is the definition of passive marine flooding?"

# Convert to embedding
query_embedding = embed_chunks(user_query)

# Convert to vector type for pgvector
query_vector = f"[{','.join(map(str, query_embedding))}]"

print(f"Query: {user_query}")
print(f"Query vector length: {len(query_embedding)}")
print(f"Query vector (first 5): {query_embedding[:5]}")

# First, let's check if we have any data in the chunks table
cur.execute("SELECT COUNT(*) FROM public.chunks")
count = cur.fetchone()[0]
print(f"Total chunks in database: {count}")

# Check the first chunk's embedding
cur.execute("SELECT layer_id, content FROM public.chunks LIMIT 1")
sample = cur.fetchone()
if sample:
    layer_id, content = sample
    print(f"Sample chunk - layer_id: {layer_id}, content: {content[:100]}...")

# Try a broader similarity search (higher threshold)
print("\n--- Testing with threshold 1.0 ---")
cur.execute("""
    SELECT content, layer_id, embedding <=> %s::vector as similarity_score
FROM public.chunks
WHERE embedding <=> %s::vector < 1.0
ORDER BY embedding <=> %s::vector
LIMIT 5""", (query_vector, query_vector, query_vector))
results = cur.fetchall()
print(f"Results with threshold 1.0: {len(results)}")
for i, result in enumerate(results):
    print(f"{i+1}. Score: {result[2]:.4f}, Layer: {result[1]}, Content: {result[0][:100]}...")

# Try without any threshold to see all similarities
print("\n--- All similarities ---")
cur.execute("""
    SELECT content, layer_id, embedding <=> %s::vector as similarity_score
FROM public.chunks
ORDER BY embedding <=> %s::vector
LIMIT 5""", (query_vector, query_vector))
all_results = cur.fetchall()
print(f"All results: {len(all_results)}")
for i, result in enumerate(all_results):
    print(f"{i+1}. Score: {result[2]:.4f}, Layer: {result[1]}, Content: {result[0][:100]}...")


## Which hazard presents the largest risk to Oahu?
## Simple Definitions (What is passive flooding?... )
## What is the greatest risk to {location}?

Query: What is the definition of passive marine flooding?
Query vector length: 1536
Query vector (first 5): [0.019133109599351883, 0.07736218720674515, 0.02231789380311966, 0.036094218492507935, 0.0024846806190907955]
Total chunks in database: 32
Sample chunk - layer_id: CRC:HI_State_80prob_{scenario}ft_SCI, content: Passive marine flooding identifies areas hydrologically connected to the ocean that would be inundat...

--- Testing with threshold 1.0 ---
Results with threshold 1.0: 5
1. Score: 0.2321, Layer: CRC:HI_State_80prob_{scenario}ft_SCI, Content: Passive marine flooding identifies areas hydrologically connected to the ocean that would be inundat...
2. Score: 0.3914, Layer: CRC:HI_State_80prob_{scenario}ft_GWI, Content: Low-lying area flooding identifies areas that are topographically below sea level rise scenarios but...
3. Score: 0.4352, Layer: CRC:HI_State_80prob_{scenario}ft_SCI, Content: These areas experience direct marine inundation as sea levels rise, with floodwater arr

In [18]:
# Assemble Context from Vector Search
context = ""

for result in results:
    context += f"{result[0]}\n" 
    context += f"Layer: {result[1]}\n"


In [19]:
# Assemble Create Prompt
prompt = f"""
You are a Hawaiian climate data assistant. Analyze the user's query and provide helpful response according to the context from the layer descriptions. Synthesize a response from the layer descriptions and the user's query.

USER QUERY: {user_query}
CONTEXT: {context}

RESPONSE FORMAT:
You MUST respond with valid JSON in this exact structure:
{{
  "type": "add_layer",
  "parameters": {{
    "layer_id": "layer_id",
    "reason": "Why adding the layer",
    "synthesized_response": "Synthesized response from the layer descriptions and the user's query"
  }}
}}
"""

In [20]:
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": prompt}],
    response_format={"type": "json_object"}
)

json_content = json.loads(response.choices[0].message.content or "{}")

In [21]:
json_content

{'type': 'add_layer',
 'parameters': {'layer_id': 'CRC:HI_State_80prob_{scenario}ft_SCI',
  'reason': 'To provide a detailed understanding of areas susceptible to passive marine flooding as defined.',
  'synthesized_response': 'Passive marine flooding refers to coastal areas that are directly connected to the ocean and are vulnerable to inundation from rising sea levels. This is determined using a modified bathtub approach that utilizes Digital Elevation Models (DEMs) and mean higher high water (MHHW) tidal datum. Such areas may become flooded as sea levels rise, particularly through surface water flow from marine waters. Unlike low-lying areas that flood indirectly without direct ocean connections, passive marine flooding specifically involves direct surface connections, making these zones critical to assess for potential marine inundation.'}}

In [63]:
json_content["parameters"]["synthesized_response"]

'Passive marine flooding is defined as the identification of areas that are hydrologically connected to the ocean and are susceptible to inundation due to rising sea levels. This is modeled using a modified bathtub approach, which incorporates Digital Elevation Models (DEMs) and Mean Higher High Water (MHHW) tidal datum to pinpoint coastal regions that fall below specific sea level heights and have open surface connections to marine waters.'

'Passive marine flooding is defined as the identification of areas that are hydrologically connected to the ocean and are susceptible to inundation due to rising sea levels. This is modeled using a modified bathtub approach, which incorporates Digital Elevation Models (DEMs) and Mean Higher High Water (MHHW) tidal datum to pinpoint coastal regions that fall below specific sea level heights and have open surface connections to marine waters.'

In [64]:
json_content["parameters"]["layer_id"]

'passive_marine_flooding'